In [2]:
import cv2
import numpy as np
from pytesseract import Output
import pytesseract
from PIL import Image

def get_contour_precedence(contour, cols):
    tolerance_factor = 80
    origin = cv2.boundingRect(contour)
    return ((origin[1] // tolerance_factor) * tolerance_factor) * cols + origin[0]

origin = cv2.imread("images/5498.jpg",0)
# origin = cv2.imread("image1.jpg",0)
img = origin.copy()
r1 = img.shape[0]*1.0/600
r2 = img.shape[1]*1.0/800

kernel33 = np.ones((3,3),dtype='uint8')
origin = cv2.resize(origin,(800,600),cv2.INTER_LINEAR)
threshOrigin = cv2.morphologyEx(origin, cv2.MORPH_BLACKHAT, kernel33)
threshOrigin = cv2.normalize(threshOrigin,  threshOrigin, 0, 255, cv2.NORM_MINMAX)
_,threshOrigin = cv2.threshold(threshOrigin,10*np.average(threshOrigin),255,cv2.THRESH_BINARY)

cv2.imshow('origin',origin)

pyra = pyra = cv2.pyrDown(origin)
s = pyra.shape

thresh = cv2.morphologyEx(pyra, cv2.MORPH_BLACKHAT, kernel33)
thresh = cv2.normalize(thresh,  thresh, 0, 255, cv2.NORM_MINMAX)
_,thresh = cv2.threshold(thresh,10*np.average(thresh),255,cv2.THRESH_BINARY)
processImage = thresh.copy()
# thresh = cv2.medianBlur(thresh, 3)
for i in range(0,s[0]-32,4):
    for j in range(0,s[1]-16,4):
        cnt = 0
        if(np.count_nonzero(thresh[i:i+16,j:j+8])>12): 
            cnt+=1
        if(np.count_nonzero(thresh[i+16:i+32,j:j+8])>12): 
            cnt+=1
        if(np.count_nonzero(thresh[i:i+16,j+8:j+16])>12): 
            cnt+=1
        if(np.count_nonzero(thresh[i+16:i+32,j+8:j+16])>12): 
            cnt+=1
        if(cnt>2):
            for x in range(i,i+32):
                for y in range(j,j+16):
                    processImage[x,y]=0
processImage = thresh - processImage
testContour = pyra.copy()
testContour = cv2.cvtColor(testContour,cv2.COLOR_GRAY2RGB)
cv2.imshow('preprocess',processImage)
processImage = cv2.dilate(processImage,np.ones((1,10)))
processImage = cv2.erode(processImage,np.ones((1,10)))
processImage = cv2.medianBlur(processImage, 7)
processImage = cv2.dilate(processImage,np.ones((1,13)))
processImage = cv2.erode(processImage,np.ones((1,13)))
_,processImage = cv2.threshold(processImage,100,255,cv2.THRESH_BINARY)


contours, hierarchy = cv2.findContours(processImage, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    # if(w > 20):
    if(w > 20 and h > 10 ):
        # try: crop = origin[y*2-5:y*2+h*2+5,x*2-5:x*2+w*2+5].copy()
        # except: crop = origin[y*2:y*2+h*2,x*2:x*2+w*2].copy()
        # crop = origin[y*2+5:y*2+h*2-5,x*2+5:x*2+w*2-5].copy()
        crop = img[int(y*2*r1):int((y*2+h*2)*r1),int(x*2*r2-5):int((x*2+w*2)*r2+5)].copy()

        if(w*1.0/h > 1.9): 
            crop = cv2.resize(crop,(390,75),cv2.INTER_LINEAR)
            h_min = 40
            h_max = 80
            longPlate = True
        else: 
            crop = cv2.resize(crop,(390,300),cv2.INTER_LINEAR)
            h_min = 95
            h_max = 150
            longPlate = False
        # crop = cv2.adaptiveThreshold(crop, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY, 199, 5)
        crop = cv2.adaptiveThreshold(crop, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY, 199, 5)

        # _,crop = cv2.threshold(crop,80,255,cv2.THRESH_BINARY)
        # crop = 255 - crop
        # crop = cv2.erode(crop,kernel33,iterations = 2)
        # # crop = cv2.morphologyEx(crop, cv2.MORPH_BLACKHAT, kernel33)
        # _,crop = cv2.threshold(crop,127,255,cv2.THRESH_BINARY)
        # crop = 255 - crop
        crop = np.pad(crop,(5,5),mode='constant',constant_values=255)
        cropCopy = crop.copy()
        cropCopy = cv2.cvtColor(cropCopy,cv2.COLOR_GRAY2RGB)

        mini_contours, hierarchy = cv2.findContours(crop, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        
        count = 0
        # first = True
        letters = []
        for contour in mini_contours:
            x,y,w,h = cv2.boundingRect(contour)
            ratio = np.count_nonzero(crop[y:y+h,x:x+w])*1.0/(h*w)
            if(h*1.0/w >= 1.1 and h > h_min and h < h_max and w > 17 and ratio < 0.7):
                cropCopy = cv2.rectangle(cropCopy, (x, y), (x + w, y + h), (36,255,12), 2)
                # cv2.imshow('contours', cropCopy)
                # cv2.waitKey(500)
                count += 1
                letters.append(contour)
                # if(first):
                #     letters = contour.copy()
                #     first = False
                # else: letters.append(contour)
            # else: cropCopy = cv2.rectangle(cropCopy, (x, y), (x + w, y + h), (0,0,255), 2)
        if(count>5):
            letters.sort(key=lambda x:get_contour_precedence(x, crop.shape[1]))
            # for debug only
            # for i in range(len(letters)):
            #     check = cv2.putText(crop, str(i), cv2.boundingRect(letters[i])[:2], cv2.FONT_HERSHEY_COMPLEX, 1, [125])
            #     cv2.imshow('check',check)
            plate = []
            cv2.imshow('contours', cropCopy)
            i=0
            for contour in letters:
                x1,y1,w1,h1 = cv2.boundingRect(contour) 
                # image = crop[y:y+h,x:x+w].copy()
                filename = 'debug/'
                filename += str(i)
                filename +='.png'
                i+=1
                saveImg = crop[y1:y1+h1,x1:x1+w1].copy()
                # saveImg = 255 - saveImg
                # saveImg = cv2.erode(saveImg,kernel33)
                # saveImg = 255 - saveImg
                # _,saveImg = cv2.threshold(saveImg,127,255,cv2.THRESH_BINARY)
                saveImg = cv2.resize(saveImg,(60,100),cv2.INTER_LINEAR)
                cv2.imwrite(filename, saveImg)
                # text = pytesseract.image_to_string(crop[y1:y1+h1,x1:x1+w1])
                # a = 1
                # letters.append(text)
                # cv2.imshow('letter',crop[y:y+h,x:x+w])
                # cv2.waitKey(0)
            i=0
            pytesseract.pytesseract.tesseract_cmd = r'Tesseract-OCR\tesseract.exe'
            for j in range(count):
                filename = 'debug/'
                filename += str(i)
                filename +='.png'
                read = Image.open(filename)
                if(i==2): text = pytesseract.image_to_string(read,lang= 'train1', config= '-c tessedit_char_whitelist=ABCDEFGHKLMNPSTUVXYZ --psm 10 --oem 3')
                elif(i==3): text = pytesseract.image_to_string(read,lang= 'train1', config= '-c tessedit_char_whitelist=0123456789ABCDEFGHKLMNPSTUVXYZ --psm 10 --oem 3')
                else: text = pytesseract.image_to_string(read,lang= 'train1', config= '-c tessedit_char_whitelist=0123456789 --psm 10 --oem 3')
                # ABCDEFGHKLMNPSTUVXYZ
                plate.append(text)
                i+=1
            print(plate)
cv2.imshow('main',processImage)
cv2.imshow('contour',testContour)




cv2.waitKey(0)
cv2.destroyAllWindows()

['5\n', '6\n', 'D\n', '0\n', '9\n', '6\n', '6\n']
